In [3]:
print('Success vs Failure')

Notebook on


In [4]:
import glob
import pandas as pd
import figs
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import chisquare
from scipy import stats
import nltk

In [5]:
import json
import figs
import glob

In [6]:
from sklearn import svm, linear_model, cluster
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neural_network import MLPClassifier,BernoulliRBM
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, balanced_accuracy_score
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn import naive_bayes, ensemble
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

In [7]:
def path2texts(path):
    
    """ Path to success or failure """
    
    texts = []
    for el in glob.glob(path):
        try:
            text = open(el).read()
            title = el.split('_')[-1]
            texts.append((title,text))
        except:
            ''
            
    return texts


def computing_features(texts):
    
    h_values = []
    ttr_values = []
    arcs = []
    
    for double in texts: 
        
        title,text = double
    
        print(title, len(text))

        words = nltk.wordpunct_tokenize(text)
        ttr = len(list(set(words)))/len(words)
        
        arc = figs.sentimarc_sid(text)
        #if len(arc)>300: 
        h = figs.get_Hurst(arc)

        arcs.append(arc)
        h_values.append(h)
        ttr_values.append(ttr)
        
    return h_values, ttr_values, arcs


def features_and_labels(h_values, ttr_values, arcs, label_name):
    labels = []
    X = []

    #n=140  #fixed n best for love stories?

    for i in range(len(h_values)):
        labels.append(label_name)
        l=len(arcs[i])
        n=int(l/6)
        X.append((h_values[i], abs(h_values[i]-0.6),
                  ttr_values[i], arcs[i][-1],
                  np.mean(arcs[i]),
                   np.mean(arcs[i][-n:]), np.mean(arcs[i][:n])
                 ))
        
    return X,labels


def path2features(path, label_name):
    
    """Feed an emotion flow path and a label name"""
    
    texts_set = path2texts(path)
    
    hs,ts,ar = computing_features(texts_set)
    
    X, labels = features_and_labels(hs,ts,ar,label_name)
    
    return X, labels

In [56]:
path_to_success = 'emotion_flow-master/data/data_all/Fiction/success/*'
path_to_failure = 'emotion_flow-master/data/data_all/Fiction/failure/*'

# 'book_eacl/Fiction/success/*' #
X_success, y_success = path2features(path_to_success, 's')

print()


#X_failure, y_faillure = path2features(path_to_failure, 'f')

if+winter+comes.txt 643882
moon+and+sixpence.txt 407464
the+lesson+of+the+master.txt 138148
the+poor+plutocrats.txt 561627
dangerous+ages.txt 361751
the+malefactor.txt 404478
the+keeper+of+the+door.txt 928092
a+knight+of+the+cumberland.txt 113183
born+again.txt 344755
youth+a+narrative.txt 71152
the+daughters+of+danaus.txt 900635
absolution.txt 525495
mcteague.txt 636751
phantastes+a+faerie+romance+for+men+and+women.txt 384826
whirligigs.txt 428528
error
debts+of+honor.txt 724401
penrod+and+sam.txt 382686
the+altar+of+the+dead.txt 80839
majorie+daw.txt 40399
roderick+hudson.txt 737166
penguin+island.txt 470067
contrary+mary.txt 468435
just+david.txt 327040
error
captain+fracasse.txt 906088
roads+of+destiny.txt 545781
ishmael+or+in+the+depths.txt 1206529
main+street.txt 947692
of+human+bondage.txt 1410680
the+alchemist's+secret.txt 188565
the+financier+a+novel.txt 1094846
the+beast+in+the+jungle.txt 101909
amaryllis+at+the+fair.txt 399875
michael+brother+of+jerry.txt 545300
cruel+as+the

In [57]:
with open('eacl_fiction_success.json', 'w') as fp:
    json.dump(X_success, fp)
    
with open('eacl_fiction_failure.json', 'w') as fp:
    json.dump(X_failure, fp)

In [58]:
import json
 
# Opening JSON file
with open('eacl_fiction_success.json') as json_file:
    X_success = json.load(json_file)
    
with open('eacl_fiction_failure.json') as json_file:
    X_failure = json.load(json_file)
    
y_success, y_failure = ['s' for el in X_success], ['f' for el in X_failure]

In [27]:
len(y_success), len(y_failure)

(81, 30)

In [40]:
X = X_success + X_failure
labels = y_success + y_failure

In [41]:
len(X) ,len(labels)

(111, 111)

In [35]:
from sklearn import svm, linear_model, cluster
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neural_network import MLPClassifier,BernoulliRBM
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, balanced_accuracy_score
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn import naive_bayes, ensemble
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import cross_val_score, KFold, ShuffleSplit, RepeatedKFold

In [36]:
classifiers_scores = {}

In [37]:
classifiers = [#linear_model.LogisticRegression,
               GaussianNB,
               LinearDiscriminantAnalysis,
               RandomForestClassifier, 
               svm.SVC,GaussianProcessClassifier,
               QuadraticDiscriminantAnalysis,
               #MLPClassifier,, BernoulliRBM,
                ]
ensemble_classifiers = [ensemble.StackingClassifier, ensemble.VotingClassifier]

In [38]:
#clf = my_clf(estimators = [('q',QuadraticDiscriminantAnalysis()), ('gaussian',GaussianNB()),('linear2', LinearDiscriminantAnalysis()), ('forest',RandomForestClassifier())])

In [39]:
for classifier in classifiers:
    
    clf = classifier()
    clf_name = clf.__str__()
    
    print(clf_name)
    
    cv_ = RepeatedKFold(n_splits=10, n_repeats=2, random_state=10)
    
    average_repeated_f_score = np.mean(cross_val_score(clf, X, labels, cv=cv_, scoring='f1_weighted', n_jobs=-1))
    average_tenfold_f_score = np.mean(cross_val_score(clf, X, labels, cv=5,scoring='f1_weighted'))
    
    X_train , X_test , y_train, y_test = train_test_split(X, labels, random_state=0, test_size=0.2)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
    single_f_score = f1_score(y_test,y_pred,average='weighted') 
    binary_f_score = f1_score(y_pred,y_test,pos_label='s',average='binary')
    
    print(single_f_score)
    
    classifiers_scores[clf_name] = [average_repeated_f_score, average_tenfold_f_score,
                                    balanced_accuracy, single_f_score, binary_f_score]

GaussianNB()


ValueError: Found input variables with inconsistent numbers of samples: [1, 111]

In [ ]:
classifiers_scores

In [82]:
clf = QuadraticDiscriminantAnalysis()

In [83]:
scores = []
for i in range(4):
    ss = cross_val_score(clf, X, labels, cv=10,scoring='f1_weighted')
    scores+=list(ss)
np.mean(scores)

/Users/au701203/opt/anaconda3/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/au701203/opt/anaconda3/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/au701203/opt/anaconda3/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/au701203/opt/anaconda3/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/au701203/opt/anaconda3/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/au701203/opt/anaconda3/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables a

0.5283295136236312

In [152]:
cv_ = RepeatedKFold(n_splits=10, n_repeats=1, random_state=10)
np.mean(cross_val_score(clf, X, labels, cv=cv_, scoring='f1_weighted', n_jobs=-1))

0.7012053082796117

In [153]:
np.mean(cross_val_score(clf, X, labels, cv=10,scoring='f1_weighted'))

0.6945867342152171

In [176]:
X_train , X_test , y_train, y_test = train_test_split(X, labels, random_state=0, test_size=0.15)

print(len(X_train), len(X_test), len(X))

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(f1_score(y_test,y_pred,average='weighted')) #balanced_accuracy_score(y_pred,y_test)
balanced_accuracy_score(y_test, y_pred)

94 17 111
0.8141923436041084


0.7115384615384616

In [177]:
len(y_pred)

17

In [178]:
print(f1_score(y_pred,y_test,pos_label='s',average='binary'))

0.888888888888889


In [81]:
for i in range(len(y_pred)): print(y_pred[i],y_test[i])

s f
s s
s s
f s
s s
f f
s s
s s
s s
s f
s f
s s
f s
f s
f s
f s
f s
f s
s f
s s
f f
s s
s s


In [28]:
### check out the balanced datasets as well!! test size=.1